In [7]:
import os
import environ
import openai
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from pathlib import Path

# 환경 변수 설정
# BASE_DIR = Path(__file__).resolve().parent.parent.parent
BASE_DIR = '/home/dongik/src/django'
print(f" [Info:] BASE_DIR: {BASE_DIR}")
env = environ.Env()
environ.Env.read_env(
    env_file=os.path.join(BASE_DIR, '.env')
)
openai.api_key = env('OPENAI_API_KEY')

# FastAPI 애플리케이션 생성
app = FastAPI()

# Pydantic 모델 - 입력 데이터 형식 정의
class TextRequest(BaseModel):
    text: str  # 요약할 텍스트

# 텍스트 요약 함수
def summarize_text_with_openai(text: str, max_tokens=150):
    """OpenAI API를 사용해 텍스트를 요약하는 함수"""
    prompt = f"다음 텍스트를 3문장으로 요약해 주세요:\n\n{text}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # 최신 GPT-3.5-turbo 모델 사용
        messages=[  
            {"role": "system", "content": "너는 문서의 텍스트를 받아서 요약해주는 역할을 해야 돼."},
            {"role": "user", "content": f"다음 텍스트를 3문장으로 요약해 주세요:\n\n{text}"},
            {"role": "user", "content": """ 형식은 아래를 참고해서 작성해줘.
                문서 제목 
               - 문장1
               - 문장2
               - 문장3
            """}
        ],
        max_tokens=max_tokens,  # 최대 토큰 수 제한
        temperature=0.7  # 창의성 정도
    )
    
    # 응답에서 요약된 텍스트 추출
    summary = response['choices'][0]['message']['content'].strip()

    return summary  

# FastAPI 엔드포인트 - POST로 텍스트 받아서 요약
@app.post("/summarize/")
async def summarize(request: TextRequest):
    try:
        # 요약 생성
        summary = summarize_text_with_openai(request.text)
        return {"summary": summary}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error in summarizing text: {str(e)}")


 [Info:] BASE_DIR: /home/dongik/src/django


In [11]:
print(summarize_text_with_openai(
    '''
넥스트레이드는 4일 오전 서울 여의도 금융투자협회센터에서 개장식을 개최하고 오전 10시부터 본격적인 운영에 들어갔다.

지난 2022년 11월 설립한 넥스트레이드는 그다음 해인 2023년 7월 예비인가를 취득한 후 올 2월 본인가를 취득하며 오늘 첫 거래를 시작했다. 장 종료 시각은 오후 8시며 내일부터는 본래대로 오전 8시부터 오후 8시까지 12시간 동안 장을 운영한다.

넥스트레이드는 이날부터 오는 16일까지 2주 동안에는 미리 공표한 10개의 종목만을 거래할 수 있다. 종목 수는 단계적으로 늘려나갈 방침이며 이달 말까지 800개의 종목을 거래할 수 있게 할 방침이다. 종목 관련 정보는 넥스트레이드 홈페이지를 통해 확인할 수 있다.

김학수 넥스트레이드 대표는 인사말에서 "넥스트레이드가 우리 자본시장의 요청에 맞춰 보다 기민하고 혁신적인 서비스를 제공할 수 있도록 최선을 다하겠다"면서 "안정적인 거래시스템 안착을 통해 우리 자본시장의 효율성 및 거래 편의성 제고 등 우리 자본시장 밸류업과 지속적인 성장에 기여하도록 노력하겠다"고 밝혔다.

이어 김병환 금융위원장은 "넥스트레이드 출범은 자본시장 거래 인프라 측면에서 밸류업으로서 굉장히 큰 의미가 있다"며 "주식 거래시간 연장으로 투자자들이 출근 전, 퇴근 이후에 여유롭게 투자가 가능해졌다"고 말했다. 

이복현 금융감독원장은 "불공정거래 감시 체계를 강화해 자본시장 전산 시스템의 안정적인 유지에 만전을 기할 것"이라고 강조했다.
'''
))

넥스트레이드의 첫 거래 개시
- 넥스트레이드는 오전 10시부터 본격적인 운영에 시작했으며, 장 종료 시각은 오후 8시로, 장 운영 시간은 오전 8시부터 오후 8시까지 12시간 동안으로 확정됐다.
- 넥스트레이드는 첫 2주 동안 10개의 종목만을 거래할 수 있으며, 종목 수는 이
